In [1]:
!unzip cifar-100_eval.zip

Archive:  cifar-100_eval.zip
   creating: cifar-100_eval/
  inflating: __MACOSX/._cifar-100_eval  
  inflating: cifar-100_eval/29_2.png  
  inflating: __MACOSX/cifar-100_eval/._29_2.png  
  inflating: cifar-100_eval/91_2.png  
  inflating: __MACOSX/cifar-100_eval/._91_2.png  
  inflating: cifar-100_eval/12_4.png  
  inflating: __MACOSX/cifar-100_eval/._12_4.png  
  inflating: cifar-100_eval/57_4.png  
  inflating: __MACOSX/cifar-100_eval/._57_4.png  
  inflating: cifar-100_eval/48_3.png  
  inflating: __MACOSX/cifar-100_eval/._48_3.png  
  inflating: cifar-100_eval/93_0.png  
  inflating: __MACOSX/cifar-100_eval/._93_0.png  
  inflating: cifar-100_eval/88_3.png  
  inflating: __MACOSX/cifar-100_eval/._88_3.png  
  inflating: cifar-100_eval/53_0.png  
  inflating: __MACOSX/cifar-100_eval/._53_0.png  
  inflating: cifar-100_eval/16_0.png  
  inflating: __MACOSX/cifar-100_eval/._16_0.png  
  inflating: cifar-100_eval/30_3.png  
  inflating: __MACOSX/cifar-100_eval/._30_3.png  
  inflating

In [2]:
%pip install torchattacks
%pip install pytorchcv
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [3]:
import os
import numpy as np
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image

import torchattacks

from tqdm import tqdm
import math
import random
import csv
import torchvision.transforms as transforms
from PIL import Image

## Useful Functions

### set seeds

In [4]:
def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## Datasets class

In [5]:
class Cifar100(Dataset):
    def __init__(self, data, labels):
        super(Cifar100, self).__init__()
        self.data = data
        self.labels = labels
        self.datasize = len(self.data)

    def __getitem__(self, idx):
        self.data[idx]
        return self.data[idx], self.labels[idx]


    def __len__(self):
        return self.datasize

## HyperParameters

In [134]:
config = {
    "batch_size": 1,
    "model_type": "ensemble",
    "attack": "DIFGSM",
    "model_name": 'resnet20_cifar100',
    "model_names": ['nin_cifar100', 'resnet20_cifar100', 'seresnet20_cifar100', 'densenet40_k12_cifar100'],
    # "model_names": ['resnet110_cifar100','resnet20_cifar100'],
    "test_model_names": ['resnet20_cifar100', 'wrn16_10_cifar100', 'ror3_164_cifar100', 'rir_cifar100', 'pyramidnet110_a270_cifar100'],
    "data_filepath": "./cifar-100_eval",
    "current_dir": "./",
    "save_dir": "adv_imgs",
    "output_filepath": "./adv_imgs",
    "steps": 64,
    "epsilon": 8 / 255,
    "alpha" : 0.125 / 255,
    "decay": 0.9,
    "seeds": 10901036
}
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Load datasets

In [135]:
same_seeds(config["seeds"])
eval_files = sorted([os.path.join(config["data_filepath"],x) for x in os.listdir(config["data_filepath"]) if x.endswith(".png")])
eval_imgs = torch.stack([transforms.ToTensor()(Image.open(filename)) for filename in eval_files])
eval_labels = torch.tensor([int(filename.split('/')[-1].split('_')[0]) for filename in eval_files], dtype=torch.long)

## Model(s)

### Single model

In [136]:
from pytorchcv.model_provider import get_model as ptcv_get_model
import torchattacks
adv_images = None
if config["model_type"] == "single":
    model_name = config["model_name"]
    model = ptcv_get_model(model_name, pretrained=True).to(device)
    attack = None
    if config["attack"] == "FGSM":
        attack = torchattacks.FGSM(model, eps=config["epsilon"])
    elif config["attack"] == "MIFGSM":
        attack = torchattacks.MIFGSM(model, eps=config["epsilon"], decay=config["decay"], steps=config["steps"])
    elif config["attack"] == "DIFGSM":
        attack = torchattacks.DIFGSM(model, eps=config["epsilon"], decay=config["decay"], alpha=config["alpha"], steps=config["steps"])
    elif config["attack"] == "PGD":
        attack = torchattacks.PGD(model, eps=config["epsilon"], alpha=config["alpha"], steps=config["steps"])
    adv_images = attack(eval_imgs, eval_labels)
    with open("attack_info.txt", 'w') as f:
        if config["attack"] == "FGSM":
            f.write(f"Attack: FGSM\nEpsilon: {config['epsilon']}\nModel_type: {config['model_type']}\nModel: {model_name}\n")
        elif config["attack"] == "PGD":
            f.write(f"Attack: PGD \nEpsilon: {config['epsilon']}\nAlpha: {config['alpha']}\nSteps: {config['steps']}\nModel_type: {config['model_type']}\nModel: {model_name}\n")
        elif config["attack"] == "DIFGSM":
            f.write(f"Attack: DIFGSM \nEpsilon: {config['epsilon']}\nDecay: {config['decay']}\nSteps: {config['steps']}\nModel_type: {config['model_type']}\nModel: {model_name}\n")
        elif config["attack"] == "MIFGSM":
            f.write(f"Attack: MIFGSM \nEpsilon: {config['epsilon']}\nDecay: {config['decay']}\nSteps: {config['steps']}\nModel_type: {config['model_type']}\nModel: {model_name}\n")


### Multiple models

In [137]:
class Ensemble(nn.Module):
    def __init__(self, model_names):
        super(Ensemble, self).__init__()
        self.models = nn.ModuleList([ptcv_get_model(model_name, pretrained=True).to(device) for model_name in model_names])

    def forward(self, x):
        y = [model(x) for model in self.models]
        y = torch.stack(y, dim=1).mean(dim=1)
        return y

In [138]:
if config["model_type"] == "ensemble":
    model_names = config["model_names"]
    with open("attack_info.txt", 'w') as f:
        if config["attack"] == "FGSM":
            f.write(f"Attack: FGSM\nEpsilon: {config['epsilon']}\nModel_type: {config['model_type']}\nModel: {model_names}\n")
        elif config["attack"] == "PGD":
            f.write(f"Attack: PGD \nEpsilon: {config['epsilon']}\nModel_type: {config['model_type']}\nAlpha: {config['alpha']}\nSteps: {config['steps']}\nModel_type: {config['model_type']}\nModel: {model_names}\n")
        elif config["attack"] == "MIFGSM":
            f.write(f"Attack: MIFGSM \nEpsilon: {config['epsilon']}\nModel_type: {config['model_type']}\nDecay: {config['decay']}\nSteps: {config['steps']}\nModel_type: {config['model_type']}\nModel: {model_names}\n")
        elif config["attack"] == "DIFGSM":
            f.write(f"Attack: DIFGSM \nEpsilon: {config['epsilon']}\nModel_type: {config['model_type']}\nDecay: {config['decay']}\nSteps: {config['steps']}\nModel_type: {config['model_type']}\nModel: {model_names}\n")
    ensemble_model = Ensemble(model_names)
    if config["attack"] == "FGSM":
        attack = torchattacks.FGSM(ensemble_model, eps=config["epsilon"])
    elif config["attack"] == "MIFGSM":
        attack = torchattacks.MIFGSM(ensemble_model, eps=config["epsilon"], decay=config["decay"], steps=config["steps"])
    elif config["attack"] == "PGD":
        attack = torchattacks.PGD(ensemble_model, eps=config["epsilon"], alpha=config["alpha"], steps=config["steps"])
    elif config["attack"] == "DIFGSM":
        attack = torchattacks.DIFGSM(ensemble_model, eps=config["epsilon"], decay=config["decay"], alpha=config["alpha"], steps=config["steps"])
    adv_images = attack(eval_imgs, eval_labels)


if adv_images == None:
    print("No model found.")
    exit()

## Save images

In [139]:
if not os.path.exists(config["save_dir"]):
    os.mkdir(config["save_dir"])
for adv_image, eval_file in zip(adv_images, eval_files):
    save_image(adv_image, os.path.join(config["current_dir"], config["save_dir"], eval_file.split('/')[-1]))

## Test Accuracies

In [140]:
adv_files = sorted([os.path.join(config["output_filepath"], x) for x in os.listdir(config["output_filepath"]) if x.endswith(".png")])
adv_images = torch.stack([transforms.ToTensor()(Image.open(filename)) for filename in adv_files])
adv_labels = torch.tensor([int(filename.split('/')[-1].split('_')[0]) for filename in adv_files], dtype=torch.long)

eval_datasets = Cifar100(eval_imgs, eval_labels)
adv_datasets  = Cifar100(adv_images, adv_labels)

eval_loader = DataLoader(eval_datasets, batch_size=config['batch_size'], shuffle=False, pin_memory=True)
adv_loader  = DataLoader(adv_datasets , batch_size=config['batch_size'], shuffle=False, pin_memory=True)

In [141]:
test_model_names = config["test_model_names"]
test_models = [ptcv_get_model(test_model_name, pretrained=True).to(device) for test_model_name in test_model_names]

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    acc = 0
    for img, label in tqdm(test_loader):
        img = img.to(device)
        label = label.to(device)
        with torch.no_grad():
            pred = model(img)
            if label[0] == torch.argmax(pred):
                acc += 1
    return acc / 500

for idx, test_model in enumerate(test_models):
    original_acc = predict(eval_loader, test_model, device)
    adv_acc = predict(adv_loader, test_model, device)
    print(f"original images accuracy: {original_acc}")
    print(f"adversarial images accuracy: {adv_acc}")
    print("")
    with open("attack_info.txt", 'a') as f:
        f.write(f"\nTestModel: {test_model_names[idx]}\nOriginal images accuracy: {original_acc}\nAdversarial images accuracy: {adv_acc}\n")

100%|██████████| 500/500 [00:02<00:00, 249.59it/s]


original images accuracy: 0.496
adversarial images accuracy: 0.002



100%|██████████| 500/500 [00:01<00:00, 282.18it/s]


original images accuracy: 0.978
adversarial images accuracy: 0.33



100%|██████████| 500/500 [00:13<00:00, 36.16it/s]


original images accuracy: 0.862
adversarial images accuracy: 0.138



100%|██████████| 500/500 [00:04<00:00, 110.81it/s]


original images accuracy: 0.952
adversarial images accuracy: 0.228



100%|██████████| 500/500 [00:12<00:00, 41.13it/s]

original images accuracy: 0.986
adversarial images accuracy: 0.546



In [142]:
if config["model_type"] == "single":
    os.system(f"zip -r {config['model_name']+'_'+config['attack']}.zip adv_imgs attack_info.txt")
else:
    model_names = "_".join(config["model_names"])
    os.system(f"zip -r {model_names+'_'+config['attack']}_step{config['steps']}.zip adv_imgs attack_info.txt")